# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# Dataset loading

In [4]:
dataset_name='pokemon'

In [26]:
images_dir = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
validation_percentage = 0
valid_format = 'png'

In [27]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [28]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [32]:
imgs_list

{'all': {'dir': 'all',
  'training': ['001.png',
   '002.png',
   '003.png',
   '004.png',
   '005.png',
   '006.png',
   '007.png',
   '008.png',
   '009.png',
   '010.png',
   '011.png',
   '012.png',
   '013.png',
   '014.png',
   '015.png',
   '016.png',
   '017.png',
   '018.png',
   '019.png',
   '020.png',
   '021.png',
   '022.png',
   '023.png',
   '024.png',
   '025.png',
   '026.png',
   '027.png',
   '028.png',
   '029.png',
   '030.png',
   '031.png',
   '032.png',
   '033.png',
   '034.png',
   '035.png',
   '036.png',
   '037.png',
   '038.png',
   '039.png',
   '040.png',
   '041.png',
   '042.png',
   '043.png',
   '044.png',
   '045.png',
   '046.png',
   '047.png',
   '048.png',
   '049.png',
   '050.png',
   '051.png',
   '052.png',
   '053.png',
   '054.png',
   '055.png',
   '056.png',
   '057.png',
   '058.png',
   '059.png',
   '060.png',
   '061.png',
   '062.png',
   '063.png',
   '064.png',
   '065.png',
   '066.png',
   '067.png',
   '068.png',
   '069.png',

In [35]:
import numpy as np
import lmdb
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img

In [ ]:
def write_lmdb(filename, imgs_dict):
    lmdb_env = lmdb.open(filename)
    lmdb_db = lmdb_env.begin(write=True)
    for k, v in imgs_dict:
        lmdb_db.put(k, v)
    

img_to_array(
    load_img(
        sorted_filenames[ix],
        grayscale=grayscale,
        target_size=self.target_size
        )
)

In [36]:
from utils.data_and_files.file_utils import get_file_path

In [ ]:
def store_lmdb(
             image_lists,
             category, 
             image_dir,
             target_size=(256, 256, 3),
             color_mode='rgb',
             class_mode='categorical',
             batch_size=32,
             episode_len=20,
             episode_shift=10,
             shuffle=True,
             seed=None,
             data_format=None,
             save_to_dir=None,
             save_prefix='',
             save_format='jpeg',
             dtype=K.floatx()
             ):
    if data_format is None:
        data_format = K.image_data_format()

    classes = list(image_lists.keys())

    num_class = len(classes)

    how_many_files = 0
    for label_name in classes:
        for _ in image_lists[label_name][category]:
            how_many_files += 1

    samples = how_many_files
    class2id = dict(zip(classes, range(len(classes))))
    id2class = dict((v, k) for k, v in class2id.items())
    classes = np.zeros((samples,), dtype='int32')

    #self.image_data_generator = image_data_generator
    target_size = tuple(target_size)
    if color_mode not in {'rgb', 'grayscale'}:
        raise ValueError('Invalid color mode:', color_mode,
                         '; expected "rgb" or "grayscale".')



    i = 0
    for label_name in classes:
        for j, _ in enumerate(image_lists[label_name][category]):
            classes[i] = class2id[label_name]
            img_path = get_file_path(image_lists,
                                      label_name,
                                      j,
                                      image_dir,
                                      category)
            img = img_to_array(
                            load_img(
                            img_path,
                            grayscale=color_mode=='grayscale',
                            target_size=target_size
                            )
                        )
            i += 1


In [37]:
load_img

<function keras_preprocessing.image.utils.load_img(path, grayscale=False, color_mode='rgb', target_size=None, interpolation='nearest')>

In [8]:
inputs_shape= image_size=(100, 100, 3)
batch_size = 32
latents_dim = 32
intermediate_dim = 50

In [9]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode=None
)

  INFO     | Found 662 training files


  INFO     | Found 147 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)


In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

30000

# Model's Layers definition

In [17]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [18]:
model_name = dataset_name+'AE_Dense_reconst_ell'
experiments_dir='experiments'+sep_local+model_name

In [19]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [20]:
inputs_shape=image_size

In [21]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_ell\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                16000032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                1650      
_________________________________________________________________
dense_5 (Dense)              (None, 30000)             1530000   
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100, 3)       0

  WARNING  | None


In [27]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAE_Dense_reconst_ell"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                16002910  
_________________________________________________________________
generative (Model)           (None, 100, 100, 3)       1532718   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 17,535,628
Trainable params: 17,535,558
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [28]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=6, 
    verbose=1, 
    restore_best_weights=False
)

In [30]:
ms = ModelSaver(filepath=_restore)

In [31]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_ell\\csv_dir\\pokemonAE_Dense_reconst_ell.csv'

In [32]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [33]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [34]:
import numpy as np

# Model Training

In [35]:
#ae.fit(
#    x=train_ds,
#    input_kw=None,
#    steps_per_epoch=int(1e4),
#    epochs=int(1e6), 
#    verbose=2,
#    callbacks=[ es, ms, csv_log, sg],
#    workers=-1,
#    use_multiprocessing=True,
#    validation_data=test_ds,
#    validation_steps=int(1e4)
#)

# Model Evaluation

## inception_score

In [36]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [37]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

calculating the inception_score mean ...


200it [1:42:45, 30.75s/it]

calculating the inception_score sigma ...


C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
200it [1:42:40, 30.91s/it]

inception_score mean: dask.array<mean_agg-aggregate, shape=(1,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>, sigma: dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>


## Frechet_inception_distance

In [38]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [39]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

calculating the inception images sigma ...


0it [00:00, ?it/s]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:271: PerformanceWarning: Increasing number of chunks by factor of 656
  axes=(left_axes, right_axes),
1it [05:04, 304.38s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:271: PerformanceWarning: Increasing number of chunks by factor of 656
  axes=(left_axes, right_axes),
2it [14:37, 384.91s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:271: PerformanceWarning: Increasing number of chunks by factor of 656
  axes=(left_axes, right_axes),
3it [29:01, 528.86s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:271: PerformanceWarning: Increasing number of chunks by factor of 656
  axes=(left_axes, right_axes),
4it [46:43, 688.70s/it]C:\ProgramData\Anaconda3\envs\tf02\lib\site-packages\dask\array\routines.py:271: PerformanceWarning: Increasing number of chunks by factor of 656
  axes=(left_axes, right_axes),


calculating the inception images mean ...


10it [00:06,  1.61it/s]

calculating the generated images sigma ...


KeyError: 'inference_inputs'

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)